In [3]:
import random
from copy import deepcopy

colors = ["green", "red", "blue", "yellow", "purple"]
color_counts = {"green": 8, "red": 7, "blue": 7, "yellow": 7, "purple": 7}

def init_deck():
    deck = []
    for color in colors:
        deck += [color] * color_counts[color]
    random.shuffle(deck)
    return deck

def deal_hands(deck):
    # Mỗi bot 14 lá
    hand1 = [deck.pop() for _ in range(14)]
    hand2 = [deck.pop() for _ in range(14)]
    return hand1, hand2

def get_hang1_range(board):
    hang1_pos = [int(pos.split('-')[1]) for pos in board if pos.startswith('1-')]
    if not hang1_pos:
        return None, None
    return min(hang1_pos), max(hang1_pos)

def legal_moves(board, hand):
    moves = []
    # ---- Hàng 1 ----
    min1, max1 = get_hang1_range(board)
    cnt_hang1 = sum([1 for pos in board if pos.startswith('1-')])
    if cnt_hang1 == 0:
        for color in set(hand):
            moves.append(('1-7', color))
    else:
        # Mở rộng trái
        if cnt_hang1 < 7 and min1 > 1:
            for color in set(hand):
                moves.append((f"1-{min1-1}", color))
        # Mở rộng phải
        if cnt_hang1 < 7 and max1 < 13:
            for color in set(hand):
                moves.append((f"1-{max1+1}", color))
        # ---- Hàng 2..7 ----
        for row in range(2, 8):  # Hàng 2 đến 7
            for col in range(1, 14 - row + 1):
                pos = f"{row}-{col}"
                below_left = f"{row-1}-{col}"
                below_right = f"{row-1}-{col+1}"
                if below_left in board and below_right in board and pos not in board:
                    for color in set(hand):
                        if board[below_left] == color or board[below_right] == color:
                            moves.append((pos, color))
    return moves

def print_board7(board):
    for row in range(1, 8):
        line = []
        for col in range(1, 14 - row + 1):
            pos = f"{row}-{col}"
            line.append(board[pos][0] if pos in board else '.')
        print(f"Hàng {row}: " + ' '.join(line))

In [6]:
deck = init_deck()
hand1, hand2 = deal_hands(deck)
hands = [hand1, hand2]
board = {}

print("=== Ván test bắt đầu (7 hàng) ===")
turn = 0
move_count = 0

while True:
    # Kiểm tra điều kiện dừng: cả 2 bot đều hết bài hoặc đều không còn nước đi hợp lệ
    moves0 = legal_moves(board, hands[0]) if hands[0] else []
    moves1 = legal_moves(board, hands[1]) if hands[1] else []
    if (not hands[0] or not moves0) and (not hands[1] or not moves1):
        print("Cả 2 bot đều không còn nước đi hợp lệ hoặc hết bài!")
        break

    moves = legal_moves(board, hands[turn]) if hands[turn] else []
    if not moves:
        print(f"Bot {turn} không còn nước đi hợp lệ hoặc hết bài, skip lượt.")
        turn = 1 - turn
        continue

    # Nước đi đầu tiên cố định là 1-7
    if move_count == 0:
        move = ('1-7', random.choice(hands[turn]))
    # Nước đi thứ hai chỉ được chọn 1-8 hoặc 1-9
    elif move_count == 1:
        moves_2 = [m for m in moves if m[0] in ('1-8', '1-9')]
        move = random.choice(moves_2) if moves_2 else random.choice(moves)
    else:
        move = random.choice(moves)
    pos, color = move
    board[pos] = color
    hands[turn].remove(color)
    print(f"\nBot {turn} đánh: {pos} [{color}]")
    print_board7(board)
    print(f"Hand Bot 0: {sorted(hands[0])}")
    print(f"Hand Bot 1: {sorted(hands[1])}")
    print('-'*42)
    turn = 1 - turn
    move_count += 1

=== Ván test bắt đầu (7 hàng) ===

Bot 0 đánh: 1-7 [green]
Hàng 1: . . . . . . g . . . . . .
Hàng 2: . . . . . . . . . . . .
Hàng 3: . . . . . . . . . . .
Hàng 4: . . . . . . . . . .
Hàng 5: . . . . . . . . .
Hàng 6: . . . . . . . .
Hàng 7: . . . . . . .
Hand Bot 0: ['blue', 'blue', 'green', 'green', 'purple', 'purple', 'red', 'red', 'red', 'red', 'yellow', 'yellow', 'yellow']
Hand Bot 1: ['blue', 'blue', 'blue', 'blue', 'green', 'purple', 'purple', 'purple', 'red', 'red', 'yellow', 'yellow', 'yellow', 'yellow']
------------------------------------------

Bot 1 đánh: 1-8 [blue]
Hàng 1: . . . . . . g b . . . . .
Hàng 2: . . . . . . . . . . . .
Hàng 3: . . . . . . . . . . .
Hàng 4: . . . . . . . . . .
Hàng 5: . . . . . . . . .
Hàng 6: . . . . . . . .
Hàng 7: . . . . . . .
Hand Bot 0: ['blue', 'blue', 'green', 'green', 'purple', 'purple', 'red', 'red', 'red', 'red', 'yellow', 'yellow', 'yellow']
Hand Bot 1: ['blue', 'blue', 'blue', 'green', 'purple', 'purple', 'purple', 'red', 'red', 'yel